In [ ]:
import numpy as np
import cv2
import os
import open3d as o3d

In [ ]:
params_dir = os.getcwd()+'/camera_params/stereo_camera_params/' #Calibration -> Parallel image 
print(params_dir)
# Load camera parameters
ret = np.load(params_dir+'ret.npy')
K = np.load(params_dir+'K.npy')
dist = np.load(params_dir+'dist.npy')
focal_length = (K[0,0] + K[1,1])/2

print(K)

In [ ]:
def downsampling_image(image, reduction_scale) : # Reduce image resolution Pyr lelel
    for i in range(0, reduction_scale) :
        if len(image.shape) > 2:
            row, col = image.shape[:2]
        else :
            row, col = image.shape
        
        image = cv2.pyrDown(image,dstsize=(col//2,row//2))
    return image

In [ ]:
#Declare stereo matching variables
#This section student has to do fine tunes by yourself for max_disp
win_size = 5
min_disparity = -1
max_disparity = 63
num_disparity = max_disparity - min_disparity #Must be divisible by 16.

<h1 style = "text-align: center">
    <a href="https://docs.opencv.org/4.5.3/d2/d85/classcv_1_1StereoSGBM.html">Stereo SemiGlobal Matching (StereoSGBM)</a>
</h1>

In [ ]:
stereo = cv2.StereoSGBM_create(
    minDisparity = min_disparity, 
    numDisparities = num_disparity,
    blockSize = 10,
    uniquenessRatio = 1,
    speckleWindowSize = 5,
    speckleRange = 5,
    disp12MaxDiff = 2,
    P1 = 8*3*win_size**2,
    P2 = 32*3*win_size**2
    )

<h1 style = "text-align: center">
    Camera Projection matrix
</h1>

In [ ]:
left_cap = cv2.VideoCapture('./videos/stereo/left_output.avi')
right_cap = cv2.VideoCapture('./videos/stereo/right_output.avi')

ret ,left_img = left_cap.read() # You have to read left and right together for syncing the video frame
ret ,right_img = right_cap.read() 

h, w = left_img.shape[:2]

R = np.float64([
    [1,0,0],
    [0,1,0],
    [0,0,1]    
])


t = np.float64([
    [0.1],
    [0.0],
    [0.0]
])

R1, R2, P1, P2, Q, validPixROI1, validPixROI2 = cv2.stereoRectify(K, dist, K, dist, (h, w), cv2.Rodrigues(R)[0], t, flags=0)
print(Q)

#Perspective transformation matrix. This one is coming from https://ags.cs.uni-kl.de/fileadmin/inf_ags/3dcv-ws14-15/3DCV_lec01_camera.pdf
Q2 = np.float32([
    [1,  0,  0,  -w/2.0],
    [0, 1,  0,  -h/2.0],
    [0,  0,  -focal_length/1000.0, 0],
    [0,  0,  0,  1]
])
print('Modified Perspective Projection matrix')
print(Q2)

<h1 style = "text-align: center">
    <a href="http://www.open3d.org/">Open3D</a>
</h1>
<h2 style = "text-align: center"> Open3D is the open-source software library for 3D data processing </h2>

In [ ]:
init =False

# Open3D Visualizer
pointcloud_visualizer = o3d.visualization.Visualizer()
pointcloud_visualizer.create_window(window_name = "Pointclouds",width = 864, height = 480)
o3d_pointclouds = o3d.geometry.PointCloud()
pointcloud_visualizer.add_geometry(o3d_pointclouds)
# Open3D Visualizer

#Creating video capture devices
left_cap = cv2.VideoCapture('./videos/stereo/left_output.avi')
right_cap = cv2.VideoCapture('./videos/stereo/right_output.avi')

ret ,left_img = left_cap.read() # You have to read left and right together for syncing the video frame
ret ,right_img = right_cap.read() 

h, w = left_img.shape[:2]

#Get the new optimal camera matrix for better undistortion
new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(K, dist, (w,h),1, (w,h))


while (left_cap.isOpened() and right_cap.isOpened() ) :
    
    ret_left, left_img = left_cap.read()
    ret_right, right_img = right_cap.read()
    
    if ret_left and ret_right :

        left_img = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
        right_img = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)

        #Undistort image
        left_img_undistorted = cv2.undistort(left_img, K, dist, None, new_camera_matrix)
        right_img_undistorted = cv2.undistort(right_img, K, dist, None, new_camera_matrix)
        
        #Downsampling each image 1 pyramid level in order to improve computational speed
        left_img_down = downsampling_image(left_img_undistorted,1)
        right_img_down = downsampling_image(right_img_undistorted,1)

        disparity_map = stereo.compute(left_img_down, right_img_down) # Range 0-2 -> 8bits 0-255
        norm_disp = cv2.normalize(disparity_map, None , alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)


        #Reproject point into 3D/Pointclouds
        pointclouds = cv2.reprojectImageTo3D(disparity_map, Q2)
        #Filter out the point with value 0 or no depth
        filtered_map = disparity_map > 0

        filtered_clouds = pointclouds[filtered_map]
        R = np.float64([
            [1,0,0],
            [0,-1,0],
            [0,0,-1]
        ])
        
        transformed_cloud = np.dot(R, filtered_clouds.T).T
        # print(transformed_cloud.shape)
        # print('max: {}'.format(transformed_cloud.max()))
        # print('min: {}'.format(transformed_cloud.min()))
        o3d_pointclouds.points = o3d.utility.Vector3dVector(transformed_cloud)
        o3d_pointclouds.remove_non_finite_points()
        
        if not init :
            pointcloud_visualizer.add_geometry(o3d_pointclouds)
            init = True
        else :
            pointcloud_visualizer.update_geometry(o3d_pointclouds)
        
        pointcloud_visualizer.poll_events()
        pointcloud_visualizer.update_renderer()
        row, col = norm_disp.shape[:2]
        norm_disp = cv2.pyrUp(norm_disp,dstsize=(col*2, row*2))
        dist_hsv = cv2.applyColorMap(norm_disp,cv2.COLORMAP_JET)
        # Depth estimation base line is 0.1 metre
        z = (focal_length * 0.1)/disparity_map
        
        
        
        left_right = cv2.hconcat([left_img, right_img])
        cv2.imshow('Frame', left_right)
        cv2.imshow('Disparity', dist_hsv)
        
        
        if cv2.waitKey(33) & 0xff == 27 :
            break

cv2.destroyAllWindows()
left_cap.release()
right_cap.release()
pointcloud_visualizer.destroy_window()